# 层次softmax(Hierarchical Softmax）

softmax制作log线性分类器，每个词的分布概率为：$$p(w_j|W_I) = y_i = \frac{exp(u_j)}{\sum _{j'=1}^{V}exp(u_j)} \tag{公式1}$$
从上面的公式（1）可以看出，softmax分母那项归一化，每次需要计算所有的V的输出值，才可以得到当前j节点的输出，当V很大的时候，$O( V )$ 的计算代价会非常高。为了降低计算代价，可以通过构建赫夫曼tree来做层级softmax，从复杂度$O(V)$降低到$O ( log_2 V )$

## 霍夫曼tree
Hierarchical softmax怎么通过赫夫曼tree来构建呢？如下图所示：<img src = "1.png" width = "600" height = "500"><center> 图1 </center>
首先对所有在$V$词表的词，根据词频来构建赫夫曼tree，词频越大，路径越短，编码信息更少。tree中的所有的叶子节点构成了词$V$，中间节点则共有$V-1$个，上面的每个叶子节点存在唯一的从根到该节点的path，如图(1)所示， 词$w_2$的path ${n(w_2,1), n(w_2,2), n(w_3,3)}$其中$n(w,j)$表示词w的path的第j个节点。


## 叶子节点词的概率表示
上图假设我们需要计算$w_2$的输出概率，我们定义从根节点开始，每次经过中间节点，做一个二分类任务（左边或者右边），所以我们定义中间节点的n左边概率为 $$ p(n,left) = \sigma (v_{n}'^{T}.h) \tag{公式2}$$
其中$v_n'$就是中间节点，由此计算右边的概率为：$$ p(n,right) = 1-\sigma (v_{n}'^{T}.h) = \sigma (-v_{n}'^{T}.h) \tag{公式3}$$
从根节点到$w_2$，计算概率值为：
$$p(w_2 = w_O)= p(n(w_2,1),left).p(n(w_2,2),left).p(n(w_3,3),right) = \sigma (v_{n(w_2,1)}'^T .h).\sigma (v_{n(w_2,2)}'^T .h). \sigma (v_{n(w_3,3)}'^T .h) \tag{公式4} $$
其中$\sigma $为sigmoid函数。
从以上公式可以推导出：$$\sum _{i=1}^V p(w_i = w_O) = 1 \tag{公式5}$$
即每次预测叶子节点的概率之和为1。

## 训练hierarchical softmax
### 1、预处理：构建Huffman数
根据语料中的每个word的词频构建赫夫曼tree，词频越高，则离树根越近，路径越短。构建完赫夫曼tree后，每个叶子节点都有唯一的路径和编码，hierarchical softmax与softmax不同的是，在hierarchical softmax中，不对V VV中的word词进行向量学习，而是对中间节点进行向量学习，而每个叶子上的节点可以通过路径中经过的中间节点去表示。
### 2、模型输入
输入部分，在cbow或者skip-gram模型，要么是上下文word对应的id词向量平均，要么是中心词对应的id向量，作为hidden层的输出向量
### 3、样本label
不同softmax的是，每个词word对应的是一个V大小的one-hot label，hierarchical softmax中每个叶子节点word，对应的label是赫夫曼编码，一般长度不超过$log_2 V $,在训练的时候，每个叶子节点的label统一编码到一个固定的长度，不足的可以进行pad
### 4、训练过程
用一个例子来描述，假如训练样本如下:  

input|output  
:-|-:  
chupacabra[0,0,0,…,1,0,0…,0] | active[1,0,1]  

假设用skip-gram模型，则第一部分，根据"chupacabra" 词的one-hot编码乘以W WW权重矩阵，得到“chupachabra”的词向量表示，也就是hidden的输出，根据目标词“active”从赫夫曼tree中得到它的路径path，即经过的节点（6，4，3），而这些中间节点的向量是模型参数需要学习的，共有V−1个向量，通过对应的节点$i_d$，取出相应的向量，假设是$w^{'}（3{*}N)$（这里设词向量维度为N），分别与hidden输出向量相乘，再经过sigmoid函数，得到一个3*1的score分值，与实际label [1,0,1], 通过如下公式：
$$path_-score=tf.log(tf.multiply(label,score)+tf.multiply(1−label,1−score))\tag{公式6}$$
$$loss=tf.reduce_-sum(tf.negative(path_-score))\tag{公式7}$$

$tf.multiply(label,score)$点乘，是获取该叶子节点路径中走右边各分支对应的概率分值，$tf.mulitply(1−label,1−score)$获取路径中走左边各分支对应的概率分值，相加就是该路径[1,0,1]对应的[右分支—左分支—右分支]分别对应的概率分值$[ s c o r e 1 , s c o r e 2 , s c o r e 3 ] $由于小于1的分值相乘，会容易溢出，则取log，乘法变加法，loss则是与我们期望的概率值越大相反，需要取负用$tf.negative$实现，计算出loss，则就可以用优化器来计算模型中各参数的梯度进行更新了
### 5、模型预测
训练过程中，每个word我们都提前知道了赫夫曼编码，所以训练的时候我们平均大概只要计$log_2V$的中间节点，就可以得到结果。但是在预测阶段，由于我们需要计算每个叶子节点输出概率值，然后取最大的一个概率，所以在预测阶段并不能省时间。那么怎么计算每个叶子节点也就是word的概率值？
从输入层到hidden层，得到输出向量，然后分别计算与每个叶子节点score分值。与训练过程一样，hidden向量与叶子节点的path经过的中间节点向量相乘，再sigmoid，然后用如下公式:
$$path_-score=tf.log(tf.multiply(label,score)+tf.multiply(1−label,1−score))\tag{公式6}$$
$$score=tf.reduce_-sum(tf.negative(path_-score))\tag{公式8}$$
计算所有叶子节点的score分值，取分值最大的就是预测的Word label
